#Данный скрипт позволяет изучать динамику количества игр, устанавливаемых каждым игроком. Т.е. можно узнать, сколько игр установил каждый игрок в тот или иной месяц, а также суммарное количество установленных к данному месяцу игр.

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime
import warnings
warnings.simplefilter('ignore')

In [ ]:
!pip install clickhouse_driver

In [ ]:
from sqlalchemy import create_engine
from clickhouse_driver import Client
from sqlalchemy.exc import IntegrityError
client = Client(host='', port='', user='', password='', database='')
engine = create_engine('')

In [ ]:
print(f'версия pandas: {pd.__version__}')
print(f'версия numpy: {np.__version__}')
print(f'версия scipy: {scipy.__version__}')

версия pandas: 1.1.5
версия numpy: 1.19.5
версия scipy: 1.4.1


## Функции для выгрузки данных

In [ ]:
def min_timestamp_upload(platform):

  """Позволяет выгрузить данные о самом первом инсталле для каждого advertising_id.
  
  Paramters:
  ----------
  platform: str, in ['ios', 'android']
    Название платформы.
  
  Returns:
  -------
  min_timestamp_data: dataframe.
    Датафрейм со временем самой первой установки для каждого advertising_id."""
  sql_query = f"""select min(user_first_touch_timestamp), 
                    JSONExtractString(device, 'advertising_id')
                    from {platform}1
                    where length(JSONExtractString(device, 'advertising_id')) > 10
                    group by JSONExtractString(device, 'advertising_id')
                union all
                select min(user_first_touch_timestamp), 
                    JSONExtractString(device, 'advertising_id')
                    from {platform}2
                    where length(JSONExtractString(device, 'advertising_id')) > 10
                    group by JSONExtractString(device, 'advertising_id')      
  """
  #т.к. платформе android соответствуют 3 таблицы (android1, android2, android3),
  # то добавим третью таблицу к запросу
  if platform == 'android':
    add_query = """union all
                select min(user_first_touch_timestamp), 
                    JSONExtractString(device, 'advertising_id')
                    from android3
                    where length(JSONExtractString(device, 'advertising_id')) > 10
                    group by JSONExtractString(device, 'advertising_id')"""
    sql_query += add_query

  result, columns = client.execute(sql_query, with_column_types=True)
  min_timestamp_data = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

  return min_timestamp_data

In [ ]:
def main_data_upload(platform,  games_type, month, year):
  """Выгрузка основных данных.
  
  Paramters:
  ---------
  platform: str, in ['ios', 'android']
  games_type: str, in ['free2play', 'premium']
  month, year: int
    Месяц и год, для которых делается выгрузка данных.
  
  Returns:
  -------
  main_data: dataframe
    Датафрейм с данными о количестве установок у каждого пользователя
    за месяц month и год year для платформы platform и типа игры games_type."""
  
  assert platform in ['android', 'ios'], f"""Вы указали название платформы '{platform}'', а надо либо 'ios' либо 'android'"""
  assert games_type in ['free2play', 'premium', 'all'], f"""Вы указали тип игр '{games_type}'', а надо либо 'free2play' либо 'premium' либо 'all'"""
  
  # укажем дополнительную часть запроса sql, которая нужна будет для выбора типа игры (free2play, premium)
  if games_type == 'free2play':
    sql_games_type_add = """ and app_info__id like '%free2play%'"""
  elif games_type == 'premium':
    sql_games_type_add = """ and app_info__id not like '%free2play%'"""
  elif games_type == 'all':
    sql_games_type_add = ''

  sql_query = f"""select JSONExtractString(device, 'advertising_id') as advertising_id, toYear(event_date), toMonth(event_date),
          count(distinct(app_info__id)) as games, max(event_date) as last_activity
          from {platform}1 
          where event_date > '2020-02-01'
            and toMonth(event_date) == {month} and toYear(event_date) == {year}
            and length(JSONExtractString(device, 'advertising_id')) > 10
            and toMonth(user_first_touch_timestamp) == toMonth(event_date){sql_games_type_add} 
          group by JSONExtractString(device, 'advertising_id'), toYear(event_date), toMonth(event_date)
          union all
          select JSONExtractString(device, 'advertising_id') as advertising_id, toYear(event_date), toMonth(event_date),
          count(distinct(app_info__id)) as games, max(event_date) as last_activity
          from {platform}2
          where event_date > '2020-02-01'
            and toMonth(event_date) == {month} and toYear(event_date) == {year}
            and length(JSONExtractString(device, 'advertising_id')) > 10
            and toMonth(user_first_touch_timestamp) == toMonth(event_date){sql_games_type_add} 
          group by JSONExtractString(device, 'advertising_id'), toYear(event_date), toMonth(event_date)
          """
  #т.к. платформе android соответствуют 3 таблицы (android1, android2, android3),
  # то добавим третью таблицу к запросу
  if platform == 'android':
    add_query = f"""union all
          select JSONExtractString(device, 'advertising_id') as advertising_id, toYear(event_date), toMonth(event_date),
          count(distinct(app_info__id)) as games, max(event_date) as last_activity
          from android3 
          where event_date > '2020-02-01'
           and toMonth(event_date) == {month} and toYear(event_date) == {year}
           and length(JSONExtractString(device, 'advertising_id')) > 10
           and toMonth(user_first_touch_timestamp) == toMonth(event_date){sql_games_type_add} 
          group by JSONExtractString(device, 'advertising_id'), toYear(event_date), toMonth(event_date)"""
    sql_query += add_query
  result, columns = client.execute(sql_query, with_column_types=True)
  main_data = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

  return main_data

##Функции для работы с данными

In [ ]:
def preprocessing_data(main_data, platform, games_type, cumulative=False):
  """Предобаботка данных. Включает предобработку некоторых полей и добавление
  нескольких новых полей.
   
   Paramters:
   --------
   main_data: dataframe
    Данные, выгруженные из базы данных при помощи функции 
    main_data_upload
  platform: str, in ['ios', 'android']
  games_type: str, in ['free2play', 'premium']
  cumulaive: bool
    Данный параметр позволяет получить суммарное количество игр у игрока, 
    которые он установил к моменту month, year. Для автоматизации
    расчётов пока данный параметр не используется.
    
    Returns:
    --------
    need_data: pd.DataFrame.
      Датафрейм с данными о количестве установок у каждого пользователя
      за месяц month и год year для платформы platform и типа игры games_type. 
      Отличается от датафрейма, выдаваемого функцией main_data_upload тем, 
      что здесь для каждого игрока уже сложена информация, собранная из 
      разных таблиц (android1, android2 и т.д.). Также, испольуя эту функцию 
      можно получить и суммарное количество установленных к данному месяцу игр у
      каждого игрока. 
    """

  #предобработка полей
  need_data = main_data.copy(deep=True)
  need_data.columns = ['advertising_id', 'active_year', 'active_month', 'games', 'last_activity']
  need_data.last_activity = pd.to_datetime(need_data.last_activity)

  # сложим количество игр из разных таблиц (android1, android2, android3 или ios1, ios2) для каждого игрока
  need_data = need_data.groupby(['advertising_id', 'active_year', 'active_month'], as_index=False)\
                             .agg({'games': 'sum', 'last_activity': 'max'})
  need_data = need_data.sort_values(by=['advertising_id', 'active_year', 'active_month'])

  if cumulative:
    # для каждого игрока и данного месяца получим суммарное количество установленных у этого игрока игр к этому месяцу
    need_cum_data = need_data.iloc[:, [0, 1, 2, 3]].groupby(['advertising_id', 'active_year', 'active_month']).sum() \
         .groupby(level=0).cumsum().reset_index()

    return need_cum_data
    #сразу же добавим поля с пометками о платформе, типе игры, месяце и годе
  need_data['platform'] = platform
  need_data['games_type'] = games_type
  return need_data

In [ ]:
def add_columns(data):
  """Позволяет добавить поле rate к предобработанным при помощи функции 
  preprocessing_data данным.
  
  Paramters:
  ---------
  data: dataframe
    Датафрейм, вышедший из фукнции preprocessing_data"""
    
  need_data = data.groupby(['games', 'active_year', 'active_month', 'platform', 'games_type'], as_index=False)\
                                  .agg({'advertising_id': 'nunique'})

  need_data['rate'] = round(need_data.advertising_id / need_data.advertising_id.sum() * 100, 1)
  
  need_data.rename(columns={'advertising_id': 'number_of_users'}, inplace=True)

  return need_data

In [ ]:
def main(year,month):
  """Основная функция, которую нужно запустить, чтобы она
  выгрузила данные из базы данных, провела расчёты и загрузила в 
  таблицу users_games_dynamic необходимые данные о количестве игр у одного игрока.
  
  Paramters: 
  ----------
  year, month: int.
   Год и месяц, для которых необходимо провести расчёты
   
  Returns:
  --------
  result_df: pd.DataFrame
    Та же таблица, которую данная функция загружает на сервер.
    """
   
  platforms_list = ['ios', 'android']
  games_type_list = ['free2play', 'premium']

  #создадим датафрейм, в который будем помещать все данные
  result_df = pd.DataFrame(columns=['games', 'active_year', 'active_month', 'platform',
                                       'games_type', 'number_of_users', 'rate'])
  #пробежимся по каждой платформе и каждому типу игры, выгрузим и предобработаем данные и 
  #добавим в датафрейм result_df
  for platform in platforms_list:
    for games_type in games_type_list:
      #будем сразу выгружать, предобрабатывать и добавлять поле rate
      data = add_columns(preprocessing_data(main_data_upload(platform, games_type, month, year), platform, games_type))
      result_df = result_df.append(data)
  result_df.columns = ['games', 'activity_year', 'activity_month', 'platform', 'games_type', 'number_of_users', 'rate']
  result_df = result_df.loc[:, ['platform', 'games_type', 'activity_year', 'activity_month', 'games', 'number_of_users', 'rate']]

  #добавим данные из датафрейма result_df в таблицу  users_games_dynamic
  try:
    result_df.to_sql('users_games_dynamic', engine, if_exists='append', index=False)
  except IntegrityError as error:
    print('Вы пытаетесь добавить в таблицу данные, нарушающие ограничения первичного ключа')
    print(error)
  return result_df